In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import missingno as msno
import nltk
import string
import re
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

df = pd.read_csv('transformed_data.csv')
clean_text_df = pd.read_csv('clean_text.csv')
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

#df['body_len'] = df['text'].apply(lambda x: len(x) - x.count(" "))
#df['punct%'] = df['text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df.head()

,title,text,value,body punct%,title punct%,punct%,try,count_punct,count_ws,try2,body_len,twitter
0,U.S. Democratic senators seek probe into Icahn...,WASHINGTON (Reuters) - Eight Democratic senato...,True,-12000.0,-200.0,-12000.0,-12000.0,1.702237,3.055027,0.877989,5.335771,0
1,Trump considers retired General Petraeus as to...,WASHINGTON (Reuters) - Donald Trump will meet ...,True,-1500.0,-100.0,-1500.0,-1500.0,1.351067,2.005175,0.711950,3.232622,0
2,CANADA SENDS TROOPS TO U.S. BORDER To Deal Wit...,Immediately following President Trump s inaugu...,False,-3700.0,-200.0,-3700.0,-3700.0,1.493636,2.843867,0.780032,4.816449,0
3,"White House: Trump considering Powell, Taylor ...",WASHINGTON (Reuters) - President Donald Trump ...,True,-1100.0,-200.0,-1100.0,-1100.0,1.305300,2.136095,0.690253,3.433236,0
4,Unearthed 2012 Trump Interview Shows A COMPLE...,Donald Trump is a liar. He will do and say any...,False,-1100.0,-200.0,-1100.0,-1100.0,1.305300,2.214694,0.690289,3.483561,0


In [3]:
del df['try']
df['punct%']=df['try2']
del df['try2']
df.head()

,title,text,value,body punct%,title punct%,punct%,count_punct,count_ws,body_len,twitter
0,U.S. Democratic senators seek probe into Icahn...,WASHINGTON (Reuters) - Eight Democratic senato...,True,-12000.0,-200.0,0.877989,1.702237,3.055027,5.335771,0
1,Trump considers retired General Petraeus as to...,WASHINGTON (Reuters) - Donald Trump will meet ...,True,-1500.0,-100.0,0.711950,1.351067,2.005175,3.232622,0
2,CANADA SENDS TROOPS TO U.S. BORDER To Deal Wit...,Immediately following President Trump s inaugu...,False,-3700.0,-200.0,0.780032,1.493636,2.843867,4.816449,0
3,"White House: Trump considering Powell, Taylor ...",WASHINGTON (Reuters) - President Donald Trump ...,True,-1100.0,-200.0,0.690253,1.305300,2.136095,3.433236,0
4,Unearthed 2012 Trump Interview Shows A COMPLE...,Donald Trump is a liar. He will do and say any...,False,-1100.0,-200.0,0.690289,1.305300,2.214694,3.483561,0


In [4]:
#due to some majore issues with size, I will be reducing the df size
#after trial and error, 25% was found to be operable, 50%+ crashed the computer even with extra steps
#def dovide(x):
    #n=25
    #m=75
    #fir =x.tail(int(len(x)*(m/100)))
    #x = x.head(int(len(x)*(n/100)))
    #return x,fir

In [5]:
#df,df2= dovide(df)
df.shape

(34361, 10)

In [6]:
clean_text_df.head()

,title,text,value
0,"['live', 'feed', 'trump', 'rally', 'check', 'l...","['', '']",False
1,"['', 'brainwashed', 'marine', 'cry', '', 'beng...","['friday', 'former', 'president', 'bill', 'cli...",False
2,"['', 'poll', 'majority', 'american', 'hate', '...","['president', 'trump', 'approval', 'rating', '...",False
3,"['', 'whoa', 'appears', 'trump', 'leaked', 'cl...","['', 'want', 'people', 'know', 'cia', 'hacked'...",False
4,"['', 'pastor', 'drop', 'massive', 'truth', 'bo...","['familiar', 'trumpkins', '', 'idiotic', 'atte...",False


In [7]:
sum(df['twitter'])

21823

In [8]:
from sklearn.model_selection import train_test_split

# splitting and testing the body text

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df[['text', 'body_len', 'punct%']], df['value'], test_size=0.45)

In [10]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['text'])

tfidf_train = tfidf_vect_fit.transform(X_train['text'])
tfidf_test = tfidf_vect_fit.transform(X_test['text'])

X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)


X_train_vect.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,118743,118744,118745,118746,118747,118748,118749,118750,118751,118752
0,4.761314,0.810015,0.013342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.432513,0.760952,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.448970,0.814143,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.091784,0.740053,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.392535,0.893062,0.007530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X_train_vect.shape

(18898, 118755)

# Model Eval

In [12]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [13]:
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = rf_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label=False, average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 396.362 / Predict time: 247.71 ---- Precision: 0.989 / Recall: 0.989 / Accuracy: 0.988


In [14]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = gb_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label=False, average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 10720.525 / Predict time: 123.534 ---- Precision: 0.998 / Recall: 0.997 / Accuracy: 0.997


In [15]:
#So we see the values are close, but the gradient is a better option, unles the time is an issue